# Block 6 Exercise 2: finding the best parameters for predicting the fare of taxi rides
We return to our Random Forest Regression and want to automatically optimize all free parameters ...

In [0]:
import pandas as pd
import numpy as np
import folium


In [2]:
#check if notebook runs in colab
import sys
IN_COLAB = 'google.colab' in sys.modules
print('running in Colab:',IN_COLAB)
path='..'
if IN_COLAB:
  #in colab, we need to clone the data from the repo
  !git clone https://github.com/keuperj/DataScienceSS20.git
  path='DataScienceSS20'

running in Colab: True
Cloning into 'DataScienceSS20'...
remote: Enumerating objects: 586, done.
remote: Total 586 (delta 0), reused 0 (delta 0), pack-reused 586
Receiving objects: 100% (586/586), 130.66 MiB | 23.77 MiB/s, done.
Resolving deltas: 100% (240/240), done.
Checking out files: 100% (219/219), done.


In [0]:
# we load the data we have saved after wrangling and pre-processing in block I
X=pd.read_csv(path+'/DATA/train_cleaned.csv')
drop_columns=['Unnamed: 0','Unnamed: 0.1','Unnamed: 0.1.1','key','pickup_datetime','pickup_date','pickup_latitude_round3','pickup_longitude_round3','dropoff_latitude_round3','dropoff_longitude_round3']
X=X.drop(drop_columns,axis=1)
X=pd.get_dummies(X)# one hot coding
#generate labels
y=X['fare_amount']
X=X.drop(['fare_amount'],axis=1)

### Scikit Optimize
Scikit Optimize (https://scikit-optimize.github.io/stable/index.html) is a AutoML toolbox wrapped around Scikit-Learn. It allows us to use state-of-the-art automatic hyper-parameter optimization on top of our learning algorithms.   



In [4]:
# install 
!pip install scikit-optimize

     |████████████████████████████████| 81kB 5.2MB/s 


### E 2.1 Bayesian Optimization of a Random Forest Regression Model
use Bayesian Optimization with Cross-Validation (https://scikit-optimize.github.io/stable/modules/generated/skopt.BayesSearchCV.html#skopt.BayesSearchCV) to find the best regression model. Compare
* linear regression (https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression) 
* Random Forest regression (https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html#sklearn.ensemble.RandomForestRegressor)
* and SVM regression (https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html#sklearn.svm.SVR)

NOTES: this can become quite compute intensive! Hence,
* use a smaller subset of the training data to run the experiments 
* think about the range of your parameters (e.g. larger number of trees in RF or high C-values in SMV will make models expensive)
* optimize only the following parameters per model type:
    * linear: no parameters to optimize
    * RF: #trees and depth
    * SVM: C and gamma (use RBF kernel)
* parallelize -> n_jobs
* use CoLab to rum the job for up to 12h 


In [0]:
from skopt import BayesSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

In [0]:
X_train = X[X.pickup_year == 2014]
X_test = X[X.pickup_year == 2015]

y_train = y[X.pickup_year == 2014]
y_test = y[X.pickup_year == 2015]

In [0]:
#linear -> no parameters to optimze

In [0]:
bayes_rf = BayesSearchCV(RandomForestRegressor(), dict(n_estimators=[5,10,25,50]), n_jobs=8, n_iter=16)
bayes_rf.fit(X_train, y_train);

In [11]:
bayes_rf.best_score_

0.846229142188592

In [12]:
bayes_rf.best_params_

OrderedDict([('n_estimators', 50)])

In [13]:
bayes_rf.score(X_test,y_test)

0.8298129348191596

In [0]:
bayes_svm = BayesSearchCV(SVR(), dict(C=[1,2,5,10]), n_jobs=8, n_iter=16)
bayes_svm.fit(X_train, y_train);

In [18]:
bayes_svm.best_score_ # manueller Abbruch nach mehreren Stunden Laufzeit...

0.023750928077453003

In [19]:
bayes_svm.best_params_

OrderedDict([('C', 10)])

In [0]:
bayes_svm.score(X_test,y_test)